In [2]:
!pip install selenium
!pip install bs4

In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, ElementClickInterceptedException
from bs4 import BeautifulSoup
import time
import json

def parse_scholarship_detail(soup):
    # ค้นหา div ที่มี class 'markdowner' ซึ่งเป็นส่วนที่มีเนื้อหาหลักของทุน
    # ตรวจสอบ class name นี้ให้แน่ใจว่าถูกต้องตาม HTML จริงๆ
    main_content_div = soup.find('div', class_='lg:w-5/6 m-auto mt-12 markdowner fMaitree')
    
    if main_content_div:
        # ใช้ .get_text(separator='\n', strip=True) เพื่อดึงข้อความทั้งหมด
        # และจัดรูปแบบให้แต่ละ Element (เช่น p, li) ขึ้นบรรทัดใหม่
        full_text = main_content_div.get_text(separator='\n', strip=True)
        return full_text
    else:
        # หากไม่พบ div นี้ ให้คืนค่าเป็นข้อความแจ้งเตือน
        return "ไม่พบเนื้อหาหลักของทุน"

In [12]:
# ตั้งค่า options สำหรับ Chrome
options = Options()
options.add_argument('--headless')  # รัน Chrome แบบไม่แสดงหน้าต่าง
options.add_argument('--disable-gpu')
options.add_argument('--no-sandbox')

# สร้าง instance ของ Chrome WebDriver
driver = webdriver.Chrome(options=options)

try:
    # เปิดหน้าเว็บ list ทุน
    url = 'https://findstudentship.eef.or.th/scholarship?grade=ทุกระดับ&cost=ทุนทั้งหมด&genre=ทุนให้เปล่า'
    driver.get(url)

    # รอให้หน้าเว็บโหลด
    time.sleep(3)

    # # คลิก "ดูเพิ่มเติม" ซ้ำๆ จนกว่าจะไม่มีให้คลิก
    # click_next_limit = 1
    # click_times = 0
    # while True:
    #     try:
    #         load_more_button = WebDriverWait(driver, 5).until(
    #             EC.element_to_be_clickable((By.XPATH, "//button[contains(text(),'ดูเพิ่มเติม')]"))
    #         )
    #         driver.execute_script("arguments[0].click();", load_more_button)
    #         time.sleep(2)
    #         click_times += 1
    #         print(f"Clicked 'ดูเพิ่มเติม' {click_times} times")
    #     except (TimeoutException, ElementClickInterceptedException):
    #         break
    #     # if click_times >= click_next_limit:
    #     #     print("Reached click limit, stopping.")
    #     #     break

    # # รอให้เนื้อหาหลังโหลดเพิ่มเติมเสร็จ
    # time.sleep(2)

    # เก็บ list ของลิงก์ 'ไปยังแหล่งทุน'
    Datas = []

    while True:
        link_elements = driver.find_elements(By.XPATH, "//div[contains(text(),'ไปยังแหล่งทุน')]")
        total = len(link_elements)
        print(f"พบทั้งหมด {total} ทุน")

        for i in range(len(Datas), total):
            try:
                link_elements = driver.find_elements(By.XPATH, "//div[contains(text(),'ไปยังแหล่งทุน')]")
                driver.execute_script("arguments[0].click();", link_elements[i])
                time.sleep(5)

                # ดึงหน้า detail แล้วเก็บ <h1>
                detail_html = driver.page_source
                detail_soup = BeautifulSoup(detail_html, 'html.parser')
                data = parse_scholarship_detail(detail_soup)

                Datas.append(data)

                driver.back()  # กลับไปยังหน้า list
                time.sleep(3)
                print(len(Datas), "ทุนที่ดึงข้อมูลแล้ว")

            except IndexError:
                print("องค์ประกอบหายไปจาก DOM หลัง back กลับมา รีโหลดลูป")
                break  # ออกจากลูปในกรณีองค์ประกอบหาย
        else:
            break  # ถ้าลูปทำจบครบทุกตัวโดยไม่ break → ออกลูป while

    print("ผลลัพธ์:")
    print(Datas)

finally:
    driver.quit()


พบทั้งหมด 10 ทุน
1 ทุนที่ดึงข้อมูลแล้ว
2 ทุนที่ดึงข้อมูลแล้ว
3 ทุนที่ดึงข้อมูลแล้ว
4 ทุนที่ดึงข้อมูลแล้ว
5 ทุนที่ดึงข้อมูลแล้ว
6 ทุนที่ดึงข้อมูลแล้ว
7 ทุนที่ดึงข้อมูลแล้ว
8 ทุนที่ดึงข้อมูลแล้ว
9 ทุนที่ดึงข้อมูลแล้ว
10 ทุนที่ดึงข้อมูลแล้ว
ผลลัพธ์:
['ชื่อทุนการศึกษา:\n\nทุนการศึกษาพระราชทาน “นวฤกษ์”\n\nหน่วยงานให้ทุนการศึกษา:\n\nมูลนิธิช่วยนักเรียนที่ขาดแคลน ในพระบรมราชินูปถัมภ์ (ม.น.ข.)\n\nคำอธิบาย:\n\nทุนนวฤกษ์ ทุนการศึกษาพระราชทานนี้ มีความมุ่งหมายที่จะช่วยเหลือนักเรียนที่เป็นคนดี และมีความประพฤติดี แต่ขาดทุนทรัพย์ให้ได้ศึกษาต่อภายในประเทศจนกว่าจำสำเร็จการศึกษาในระดับการศึกษาตามสมควรแก่อัตภาพ เพื่อช่วยเหลือนักเรียนที่มีความประพฤติดี  ขยัน  อดทน แต่ขาดแคลนทุนทรัพย์ให้มีโอกาสได้ศึกษาต่อ สายอาชีพ ระดับ ปวช. ปวส. และระดับปริญญาตรีตามสมควรแก่ศักยภาพของผู้เรียนโดยไม่มีข้อผูกมัดใด ๆ\n\nคุณสมบัติผู้รับทุน:\n\nเป็นนักเรียน ซึ่งได้รับการคัดเลือกขั้นต้นจากสถานศึกษาของผู้สมัครขอรับทุน\n\nกำลังเรียนอยู่ชั้นมัธยมศึกษาปีที่ 6 มีผลการเรียนในปีที่ผ่านมาไม่ต่ำกว่า 2.50\n\nมีฐานะยากจน  สถานศึกษาเห็นว่าส

In [5]:
with open("scholarship_detailAll.json", 'w', encoding='utf-8') as f:
    json.dump(Datas, f, ensure_ascii=False, indent=4)